<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/notebooks/071_LLM_Optimization_KV_Cache.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚡ LLM Optimization: KV Cache & Flash Attention

Generowanie tekstu w modelach GPT jest procesem **autoregresyjnym**:
1.  Wpisujesz "Ala".
2.  Model liczy wszystko i zwraca "ma".
3.  Wpisujesz "Ala ma".
4.  Model liczy wszystko OD ZERA i zwraca "kota".

To marnotrawstwo. Obliczenia dla "Ala" w kroku 4 są identyczne jak w kroku 2.

**Rozwiązanie: KV Cache.**
Zamiast wyrzucać wektory Key i Value dla poprzednich słów, trzymamy je w pamięci (Cache).
W nowym kroku obliczamy Attention tylko dla **jednego, nowego tokena** i doklejamy go do Cache'a.

Złożoność obliczeniowa spada z $O(N^2)$ (dla całej sekwencji) do $O(N)$ (dla kroku).

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt

# Konfiguracja
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
D_MODEL = 512
NUM_HEADS = 8
HEAD_DIM = D_MODEL // NUM_HEADS
SEQ_LEN = 100 # Długość generowanego tekstu

print(f"Symulacja na: {DEVICE}")

Symulacja na: cuda


## Implementacja Standardowa (Bez Cache)

Najpierw zbudujmy "głupią" warstwę Attention, która za każdym razem przelicza całe zdanie od początku.
To jest to, co robi model podczas *treningu* (bo wtedy znamy całe zdanie), ale podczas *generowania* jest to bardzo nieefektywne.

In [2]:
class StandardAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.W_q = nn.Linear(D_MODEL, D_MODEL, bias=False)
        self.W_k = nn.Linear(D_MODEL, D_MODEL, bias=False)
        self.W_v = nn.Linear(D_MODEL, D_MODEL, bias=False)
        self.out = nn.Linear(D_MODEL, D_MODEL, bias=False)

    def forward(self, x):
        # x shape: [Batch, Seq_Len, D_Model]
        batch, seq_len, _ = x.shape
        
        # 1. Projekcje Q, K, V
        Q = self.W_q(x).view(batch, seq_len, NUM_HEADS, HEAD_DIM).transpose(1, 2)
        K = self.W_k(x).view(batch, seq_len, NUM_HEADS, HEAD_DIM).transpose(1, 2)
        V = self.W_v(x).view(batch, seq_len, NUM_HEADS, HEAD_DIM).transpose(1, 2)
        
        # 2. Scaled Dot-Product Attention
        # (Tutaj PyTorch robi to za nas wydajnie, ale i tak liczy całą macierz N*N)
        attn_output = F.scaled_dot_product_attention(Q, K, V, is_causal=True)
        
        # 3. Scalenie głowic
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch, seq_len, D_MODEL)
        return self.out(attn_output)

print("Standard Attention gotowe.")

Standard Attention gotowe.


## Implementacja z KV Cache

Teraz wersja sprytna.
Metoda `forward` przyjmuje dodatkowy argument `kv_cache`.
1.  Jeśli to pierwszy krok -> licz wszystko.
2.  Jeśli to kolejny krok -> wejście to tylko **ostatni token**.
3.  Oblicz K i V tylko dla tego tokena.
4.  Doklej do `kv_cache`.
5.  Użyj całego cache do policzenia Attention.

In [3]:
class CachedAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.W_q = nn.Linear(D_MODEL, D_MODEL, bias=False)
        self.W_k = nn.Linear(D_MODEL, D_MODEL, bias=False)
        self.W_v = nn.Linear(D_MODEL, D_MODEL, bias=False)
        self.out = nn.Linear(D_MODEL, D_MODEL, bias=False)

    def forward(self, x, kv_cache=None):
        # x shape: [Batch, 1, D_Model] (Tylko NOWY token!)
        batch, seq_len, _ = x.shape 
        
        # 1. Projekcje (tylko dla nowego tokena)
        q = self.W_q(x).view(batch, seq_len, NUM_HEADS, HEAD_DIM).transpose(1, 2)
        k = self.W_k(x).view(batch, seq_len, NUM_HEADS, HEAD_DIM).transpose(1, 2)
        v = self.W_v(x).view(batch, seq_len, NUM_HEADS, HEAD_DIM).transpose(1, 2)
        
        # 2. Obsługa Cache
        if kv_cache is not None:
            prev_k, prev_v = kv_cache
            # Doklejamy nowe k i v do starych
            k = torch.cat([prev_k, k], dim=2)
            v = torch.cat([prev_v, v], dim=2)
            
        # Zapisujemy nowy cache na przyszłość
        new_cache = (k, v)
        
        # 3. Attention
        # Q ma długość 1 (nowy token). K i V mają długość całej historii.
        # Dzięki temu nowy token "patrzy" na wszystkich poprzedników.
        attn_output = F.scaled_dot_product_attention(q, k, v, is_causal=False) # Causal niepotrzebny, bo Q to 1 token
        
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch, seq_len, D_MODEL)
        return self.out(attn_output), new_cache

print("Cached Attention gotowe.")

Cached Attention gotowe.


## Wyścig: Generowanie Tekstu

Zasymulujemy proces generowania 100 kolejnych tokenów.
*   **Standard:** W każdej pętli podajemy całą historię (`input_ids`).
*   **Cached:** W każdej pętli podajemy tylko nowy token i cache.

Zmierzymy czas.

In [4]:
# Inicjalizacja modeli
model_std = StandardAttention().to(DEVICE).eval()
model_cached = CachedAttention().to(DEVICE).eval()

# Dummy input (Startujemy od 10 tokenów)
initial_input = torch.randn(1, 10, D_MODEL).to(DEVICE)

# --- TEST 1: STANDARD ---
start_time = time.time()
current_input = initial_input.clone()

for _ in range(SEQ_LEN):
    with torch.no_grad():
        # Musimy podać CAŁĄ historię
        out = model_std(current_input)
        # Bierzemy ostatni wektor jako "nowy token" (symulacja)
        next_token = out[:, -1:, :] 
        # Doklejamy do wejścia
        current_input = torch.cat([current_input, next_token], dim=1)

time_std = time.time() - start_time
print(f"Standard Time: {time_std:.4f} s")


# --- TEST 2: KV CACHE ---
start_time = time.time()
current_token = initial_input[:, -1:, :].clone() # Tylko ostatni na start
# Najpierw musimy "napełnić" cache historią (Prefill phase)
# W uproszczeniu: pomijamy prefill i zakładamy, że startujemy od zera albo robimy pass na initial_input
# Zróbmy poprawny start:
cache = None
# 1. Prefill (przetwarzamy prompt)
with torch.no_grad():
    _, cache = model_cached(initial_input, kv_cache=None)

# 2. Generation loop
for _ in range(SEQ_LEN):
    with torch.no_grad():
        # Podajemy TYLKO ostatni token i cache
        out, cache = model_cached(current_token, kv_cache=cache)
        next_token = out # To już jest tylko 1 token
        current_token = next_token

time_cache = time.time() - start_time
print(f"Cached Time:   {time_cache:.4f} s")

speedup = time_std / time_cache
print(f"🚀 Przyspieszenie: {speedup:.2f}x")

Standard Time: 0.1483 s
Cached Time:   0.0291 s
🚀 Przyspieszenie: 5.09x


## Czym jest Flash Attention? (Teoria)

Przyspieszyliśmy obliczenia (mniej FLOPs). Ale jest jeszcze problem **pamięci (VRAM)**.
Przy długich sekwencjach (np. 100k tokenów), macierz uwagi $N \times N$ nie mieści się w pamięci GPU.

**Flash Attention (2022/2023):**
To inżynierski majstersztyk na poziomie sprzętowym (CUDA).
GPU ma dwa rodzaje pamięci:
1.  **HBM (High Bandwidth Memory):** Wielka, ale wolna (jak lodówka w kuchni).
2.  **SRAM (Static RAM):** Malutka, ale superszybka (jak deska do krojenia).

Tradycyjne Attention ciągle przenosi macierze z HBM do SRAM i z powrotem.
**Flash Attention** używa techniki **Tiling (Kafelkowanie)**. Dzieli macierz na małe klocki, które mieszczą się w SRAM, liczy wszystko "na desce do krojenia" i odsyła do "lodówki" tylko gotowy wynik.

Dzięki temu jest:
1.  Szybsze (mniej czekania na dane).
2.  Liniowe pamięciowo (nie tworzy gigantycznej macierzy $N \times N$).

W PyTorch 2.0+ funkcja `F.scaled_dot_product_attention` automatycznie używa Flash Attention, jeśli masz odpowiednią kartę graficzną!

## 🧠 Podsumowanie: Dlaczego to kluczowe?

1.  **KV Cache** jest obowiązkowe przy generowaniu tekstu (Inference). Bez tego ChatGPT generowałby jedno zdanie minutę.
    *   *Koszt:* Zużywa VRAM (pamięć karty) na przechowywanie kluczy i wartości. Im dłuższa rozmowa, tym więcej pamięci zajmuje cache.
2.  **Flash Attention** pozwala na obsługę **długich kontekstów** (np. GPT-4 Turbo 128k, Claude 200k). Bez tego macierz uwagi po prostu by się nie zmieściła w pamięci.

Jako inżynier AI, musisz wiedzieć, że **"Memory is the bottleneck"**. Większość nowoczesnych optymalizacji (PagedAttention w vLLM) polega na lepszym zarządzaniu KV Cachem.